# SOAP example

To install rascal:
(NOTE: See the top-level README for the most up-to-date installation instructions.)
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

In [1]:
import sys
import time
sys.path.insert(0,'../build/')
sys.path.insert(0,'../build/bindings/')
import rascal
import rascal.lib as lrl
import json
import ase
from ase.visualize import view
import numpy as np
import sys

In [2]:
test_hypers = {"interaction_cutoff": 4.0,
               "cutoff_smooth_width": 0.0, 
               "max_radial": 8,
               "max_angular": 6,
               "gaussian_sigma_type": "Constant",
               "gaussian_sigma_constant": 0.3}

nmax = test_hypers["max_radial"]
lmax = test_hypers["max_angular"]

In [11]:
start = time.time()

frames = ase.io.read('../tests/reference_data/dft-smiles_500.xyz',':')

with lrl._rascal.utils.ostream_redirect():
    sph_expn = rascal.representation.SphericalExpansion(**test_hypers)

with lrl._rascal.utils.ostream_redirect():
    expansions = sph_expn.transform(frames)
    
X = expansions.get_feature_matrix().T
ncen = len(X)
coeffs = np.zeros((ncen,nmax,lmax+1,2*lmax+1),float)
for icen in range(ncen):
    ii=0
    for l in range(lmax+1):
        for im in range(2*l+1):
            for n in range(nmax):
                coeffs[icen,n,l,im] = X[icen,ii]
                ii+=1

size = nmax*nmax*(lmax+1)
power = np.zeros((ncen,size),float)
for icen in range(ncen):
    jj = 0
    for n1 in range(nmax):
        for n2 in range(nmax):
            for l in range(lmax+1):
                power[icen,jj] = np.dot(coeffs[icen,n1,l],coeffs[icen,n2,l])/np.sqrt(2*l+1)
                jj+=1
    power[icen] /= np.linalg.norm(power[icen])

kernel = np.dot(power,power.T)

print(time.time()-start)

13.21003007888794


array([[1.        , 0.78498309, 0.77102096, 0.7998807 , 0.84086421,
        0.81529901, 0.95384259, 0.84304326, 0.84244704, 0.88122679,
        0.88476097],
       [0.78498309, 1.        , 0.7903963 , 0.99063544, 0.85978093,
        0.88773657, 0.87112679, 0.77097342, 0.77354254, 0.82301577,
        0.81633501],
       [0.77102096, 0.7903963 , 1.        , 0.848918  , 0.94153579,
        0.94210724, 0.82232767, 0.88278817, 0.88740778, 0.84152224,
        0.84133394],
       [0.7998807 , 0.99063544, 0.848918  , 1.        , 0.89899156,
        0.92387989, 0.88664846, 0.81337362, 0.81602242, 0.84778038,
        0.84159442],
       [0.84086421, 0.85978093, 0.94153579, 0.89899156, 1.        ,
        0.9851828 , 0.9029964 , 0.95011866, 0.94365522, 0.93237547,
        0.93146923],
       [0.81529901, 0.88773657, 0.94210724, 0.92387989, 0.9851828 ,
        1.        , 0.88636077, 0.93102147, 0.93532317, 0.92528824,
        0.92189124],
       [0.95384259, 0.87112679, 0.82232767, 0.88664846, 0.

In [15]:
coeffs = np.zeros((ncen,nmax,lmax+1,2*lmax+1),float)
for icen in range(ncen):
    ii=0
    for l in range(lmax+1):
        for im in range(2*l+1):
            for n in range(nmax):
                coeffs[icen,n,l,im] = X[icen,ii]
                ii+=1

coeffs[0,:,2,:5]

[[ 1.23176816e-05  3.98402626e-04 -1.35649919e-03 -1.25934295e-04
   8.71595792e-04]
 [ 2.16284125e-04  1.53552548e-03 -4.94334495e-03 -4.83316342e-04
   3.87383042e-03]
 [-1.98236464e-04  1.55770943e-03 -8.80783921e-03 -6.07346455e-04
   6.76696575e-04]
 [ 3.67650718e-04 -3.57598282e-04 -3.11432242e-03 -6.00809766e-05
  -3.68482713e-03]
 [ 5.36952713e-05 -2.52223773e-04 -4.06587360e-03 -1.08082785e-04
  -2.38376156e-03]
 [-2.28534348e-04 -4.73937711e-04 -2.73264840e-03  1.90533041e-05
  -4.19016892e-03]
 [ 3.05471470e-04 -3.23924086e-04 -2.91311772e-03 -3.22698750e-05
  -4.08783982e-03]
 [ 1.40815790e-04 -1.45905022e-04 -1.04351150e-03 -5.48858000e-06
  -1.69451160e-03]]


In [16]:
size = nmax*nmax*(lmax+1)
power = np.zeros((ncen,size),float)
for icen in range(ncen):
    jj = 0
    for n1 in range(nmax):
        for n2 in range(nmax):
            for l in range(lmax+1):
                power[icen,jj] = np.dot(coeffs[icen,n1,l],coeffs[icen,n2,l])/np.sqrt(2*l+1)
                jj+=1
    power[icen] /= np.linalg.norm(power[icen])
kernel = np.dot(power,power.T)
kernel

array([[1.        , 0.78498309, 0.77102096, 0.7998807 , 0.84086421,
        0.81529901, 0.95384259, 0.84304326, 0.84244704, 0.88122679,
        0.88476097],
       [0.78498309, 1.        , 0.7903963 , 0.99063544, 0.85978093,
        0.88773657, 0.87112679, 0.77097342, 0.77354254, 0.82301577,
        0.81633501],
       [0.77102096, 0.7903963 , 1.        , 0.848918  , 0.94153579,
        0.94210724, 0.82232767, 0.88278817, 0.88740778, 0.84152224,
        0.84133394],
       [0.7998807 , 0.99063544, 0.848918  , 1.        , 0.89899156,
        0.92387989, 0.88664846, 0.81337362, 0.81602242, 0.84778038,
        0.84159442],
       [0.84086421, 0.85978093, 0.94153579, 0.89899156, 1.        ,
        0.9851828 , 0.9029964 , 0.95011866, 0.94365522, 0.93237547,
        0.93146923],
       [0.81529901, 0.88773657, 0.94210724, 0.92387989, 0.9851828 ,
        1.        , 0.88636077, 0.93102147, 0.93532317, 0.92528824,
        0.92189124],
       [0.95384259, 0.87112679, 0.82232767, 0.88664846, 0.